In [2]:
def str_to_lower(s):
    return s.lower()

def str_trim(s):
    return s.strip()

def date_change_format(date_str, from_format, to_format):
    from datetime import datetime
    return datetime.strptime(date_str, from_format).strftime(to_format)

# Initialization
bFound = False
iDateIndex = 0
iIAMcntr = 0

iaOnsetDTTM = []
saActTaken = []
saAllergen = []
saAnnotate = []
saIAM = []

sActTaken = ""
sAllIndex = ""
sAnnotations = ""
pTW_MRN_AlgID = f"{in_data['ALG'][0]['APM_Patient_Number']}_{in_data['ALG'][0]['AllergyID']}"
sIAMseg = ""
sReactions = ""

# MSH Segment
out.MSH.SendingApplication.NamespaceID = mshDefault_SendingApp_TW
out.MSH.SendingFacility.NamespaceID = defaultSendingFac
out.MSH.ReceivingApplication.NamespaceID = mshDefault_ReceivingApp
out.MSH.ReceivingApplication.UniversalID = mshDefault_ReceivingApp
out.MSH.ReceivingFacility.NamespaceID = mshDefault_ReceivingFac
out.MSH.ReceivingFacility.UniversalID = mshDefault_ReceivingFac

setEncodedValue(out.MSH.DateTimeOfMessage,
                date_change_format(CurrentDateTime(True), "MM/dd/yyyy HH:mm:ss", "HL7"))

out.MSH.MessageType.MessageCode = "ADT"
out.MSH.MessageType.TriggerEvent = "A60"
out.MSH.MessageControlID = pTW_MRN_AlgID
out.MSH.ProcessingID.ProcessingID = mshDefault_ProcessingID
out.MSH.VersionID.VersionID = mshDefault_VersionID

# PID Segment
out.PID.PatientIdentifierList[0].IDNumber = in_data['ALG'][0]['APM_Patient_Number']
out.PID.PatientIdentifierList[0].IdentifierTypeCode = defaultMRNID
out.PID.PatientName[0].FamilyName.Surname = in_data['ALG'][0]['APM_Patient_Last_Name']
out.PID.PatientName[0].GivenName = in_data['ALG'][0]['APM_Patient_First_Name']
out.PID.PatientName[0].MiddleInitialOrName = in_data['ALG'][0]['APM_Patient_MI']

setEncodedValue(out.PID.DateTimeOfBirth,
                date_change_format(in_data['ALG'][0]['APM_Patient_DOB'][:10], "yyyy-MM-dd", "HL7"))

out.PID.AdministrativeSex = in_data['ALG'][0]['APM_Patient_Sex']
out.PID.PatientAddress[0].StreetAddress.StreetOrMailingAddress = in_data['ALG'][0]['APM_Patient_Street1']
out.PID.PatientAddress[0].OtherDesignation = in_data['ALG'][0]['APM_Patient_Street2']
out.PID.PatientAddress[0].City = in_data['ALG'][0]['APM_Patient_City']
out.PID.PatientAddress[0].StateOrProvince = str_trim(in_data['ALG'][0]['APM_Patient_State'])
out.PID.PatientAddress[0].ZipOrPostalCode = str_trim(in_data['ALG'][0]['APM_Patient_Zip_Code'])

out.PID.HomePhoneNumber[0].PhoneNumberString_DEPRECATED_FROM_2_3 = in_data['ALG'][0]['APM_Patient_HomePhone']
out.PID.HomePhoneNumber[0].EmailAddress = in_data['ALG'][0]['APM_Patient_Email']
out.PID.BusinessPhoneNumber[0].PhoneNumberString_DEPRECATED_FROM_2_3 = in_data['ALG'][0]['APM_Patient_WorkPhone']

# IAM Segment
for i in range(len(in_data['ALG'])):
    if (in_data['ALG'][i]['Status'] == "Active" and 
        len(in_data['ALG'][i]['AllergenName']) > 0 and 
        "no known" not in str_to_lower(in_data['ALG'][i]['AllergenName'])):
        
        # Build IAM segment starting with IAM-2
        sIAMseg = f"{str_to_lower(in_data['ALG'][i]['AllergenType']).replace(' ', '')}^{in_data['ALG'][i]['AllergenType']}|"

        # Check RxNorm code or UNII code, default to allergen name
        if len(str_trim(in_data['ALG'][i]['RxNormCode'])) > 0:
            sIAMseg += f"{str_trim(in_data['ALG'][i]['RxNormCode'])}^{str_trim(in_data['ALG'][i]['AllergenName'])}^RXN|"
            sAllIndex = f"{str_trim(in_data['ALG'][i]['RxNormCode'])}-{str_trim(in_data['ALG'][i]['AllergenName'])}"
        elif len(str_trim(in_data['ALG'][i]['UNIICode'])) > 0:
            sIAMseg += f"{str_trim(in_data['ALG'][i]['UNIICode'])}^{str_trim(in_data['ALG'][i]['AllergenName'])}^UNII|"
            sAllIndex = f"{str_trim(in_data['ALG'][i]['UNIICode'])}-{str_trim(in_data['ALG'][i]['AllergenName'])}"
        else:
            sIAMseg += f"^{str_trim(in_data['ALG'][i]['AllergenName'])}^NAME|"
            sAllIndex = f"NAME-{str_trim(in_data['ALG'][i]['AllergenName'])}"

        # Reaction severity
        sIAMseg += f"{in_data['ALG'][i]['ReactionSeverity1']}|" if in_data['ALG'][i]['ReactionSeverity1'] != "No Severity" else "|"

        # Collect reactions
        sReactions = ""
        for j in range(1, 6):
            reaction = in_data['ALG'][i].get(f'Reaction{j}', "No Reactions")
            if reaction != "No Reactions":
                if len(sReactions) > 0:
                    sReactions += f"~{reaction}^{reaction}"
                else:
                    sReactions = f"{reaction}^{reaction}"
        
        sIAMseg += f"{sReactions}|A||||NTL001^{str_trim(in_data['ALG'][i]['AllergenName'])}^EXTELG|"

        # Estimated reaction date
        est_react_date = str_trim(in_data['ALG'][i]['EstReactDate'])
        if len(est_react_date) > 0 and est_react_date[:2] != "21" and est_react_date[:4] != "1900":
            sIAMseg += f"{date_change_format(est_react_date[:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd')}||"
            iDateIndex = int(date_change_format(est_react_date[:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd'))
        else:
            sIAMseg += f"{date_change_format(in_data['ALG'][i]['RecordedDTTM'][:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd')}||"
            iDateIndex = int(date_change_format(in_data['ALG'][i]['RecordedDTTM'][:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd'))

        sIAMseg += f"{date_change_format(in_data['ALG'][i]['RecordedDTTM'][:19], 'yyyy-MM-dd HH:mm:ss', 'HL7')}|"
        sIAMseg += f"{in_data['ALG'][i]['RecordedBYLastName']}^{in_data['ALG'][i]['RecordedBYFirsttName']}||||"

        if len(str_trim(in_data['ALG'][i]['RecordedBYNPI'])) > 0:
            sIAMseg += f"{str_trim(in_data['ALG'][i]['RecordedBYNPI'])}^{in_data['ALG'][i]['RecordedBYLastName']}^{in_data['ALG'][i]['RecordedBYFirsttName']}^^^^^^NPI^^^^NPI||"
        else:
            sIAMseg += f"{defaultProvID}^{defaultProvLName}^{defaultProvFName}^^^^^^{asgnAuth_OCMW_Provider}^^^^{idType_OCMW_Provider}||"

        sIAMseg += date_change_format(in_data['ALG'][i]['RecordedDTTM'][:19], "yyyy-MM-dd HH:mm:ss", "HL7")

        # Check for annotations and actions
        sAnnotations = in_data['ALG'][i]['Annotation'] if len(in_data['ALG'][i]['Annotation']) > 0 and "allergy:" not in str_to_lower(in_data['ALG'][i]['Annotation']) and in_data['ALG'][i]['Annotation'] != "NULL" else ""
        sActTaken = in_data['ALG'][i]['ActionTaken'] if len(in_data['ALG'][i]['ActionTaken']) > 0 and in_data['ALG'][i]['ActionTaken'] != "NULL" else ""

        # Check if allergen exists in saAllergen
        bFound = False
        for j in range(len(saAllergen)):
            if saAllergen[j] == sAllIndex.upper():
                if iaOnsetDTTM[j] < iDateIndex:
                    iaOnsetDTTM[j] = iDateIndex
                    saIAM[j] = sIAMseg
                    saAnnotate[j] = sAnnotations
                    saActTaken[j] = sActTaken
                bFound = True
                break

        if not bFound:
            saAllergen.append(sAllIndex.upper())
            iaOnsetDTTM.append(iDateIndex)
            saIAM.append(sIAMseg)
            saAnnotate.append(sAnnotations)
            saActTaken.append(sActTaken)
            iIAMcntr += 1

# If there are no IAM segments, set the message as invalid
if iIAMcntr == 0:
    gbValidMsg = False
else:
    # Build the IAM segments
    for i in range(len(saAllergen)):
        setEncodedValue(out.Allergens.append(), f"IAM|{i + 1}|{saIAM[i]}")
        if len(saAnnotate[i]) > 0:
            out.Allergens[-1].NTE.append({'Comment': saAnnotate[i]})
        if len(saActTaken[i]) > 0:
            out.Allergens[-1].NTE.append({'Comment': saActTaken[i]})

NameError: name 'in_data' is not defined

In [ ]:
import pandas as pd
from datetime import datetime

# Helper functions
def str_to_lower(s):
    return s.lower()

def str_trim(s):
    return s.strip()

# Updating the date handling to skip invalid dates or provide a default value
def date_change_format(date_str, from_format, to_format):
    from datetime import datetime
    # Handle NaN, empty, or invalid date values
    if pd.isna(date_str) or date_str == 'nan' or date_str == '0':
        return ''  # Return an empty string for invalid dates
    # Ensure the date is treated as a string
    if not isinstance(date_str, str):
        date_str = str(date_str)
    # Truncate fractional seconds if present
    if '.' in date_str:
        date_str = date_str.split('.')[0]
    # Correct the format for Python's strptime
    from_format = from_format.replace("yyyy", "%Y").replace("MM", "%m").replace("dd", "%d").replace("HH", "%H").replace("mm", "%M").replace("ss", "%S")
    to_format = to_format.replace("yyyy", "%Y").replace("MM", "%m").replace("dd", "%d").replace("HH", "%H").replace("mm", "%M").replace("ss", "%S")
    try:
        return datetime.strptime(date_str, from_format).strftime(to_format)
    except ValueError:
        return ''  # Return an empty string for date parsing errors

# Re-run the process with the updated handling for invalid dates
hl7_output = convert_to_hl7(df_fixed)

# Save the HL7 output to a text file
output_file_path = '/mnt/data/hl7_output_final_fixed.txt'
with open(output_file_path, 'w') as f:
    f.write(hl7_output)

output_file_path  # Returning the file path for download

# Initializing variables for processing
bFound = False
iDateIndex = 0
iIAMcntr = 0

iaOnsetDTTM = []
saActTaken = []
saAllergen = []
saAnnotate = []
saIAM = []

sActTaken = ""
sAllIndex = ""
sAnnotations = ""
sIAMseg = ""
sReactions = ""

# Assume mshDefault_SendingApp_TW, mshDefault_ReceivingApp, etc., are constants you have defined
mshDefault_SendingApp_TW = "App_TW"
defaultSendingFac = "SendingFac"
mshDefault_ReceivingApp = "ReceivingApp"
mshDefault_ReceivingFac = "ReceivingFac"
defaultMRNID = "MRN"
mshDefault_ProcessingID = "ProcessingID"
mshDefault_VersionID = "2.5"

# Processing each row in the DataFrame (replace df_fixed with your DataFrame)
for row_index, row in df_fixed.iterrows():
    
    # MSH Segment
    out = {}  # Initialize your output object for HL7
    out['MSH'] = {}
    out['MSH']['SendingApplication'] = mshDefault_SendingApp_TW
    out['MSH']['SendingFacility'] = defaultSendingFac
    out['MSH']['ReceivingApplication'] = mshDefault_ReceivingApp
    out['MSH']['ReceivingFacility'] = mshDefault_ReceivingFac

    # Setting the DateTime of the message
    out['MSH']['DateTimeOfMessage'] = date_change_format(row['CreateDTTM'], "yyyy-MM-dd HH:mm:ss", "yyyyMMddHHmmss")

    # PID Segment
    out['PID'] = {}
    out['PID']['IDNumber'] = row['APM_Patient_ID']
    out['PID']['IdentifierTypeCode'] = defaultMRNID
    out['PID']['FamilyName'] = row['APM_Patient_Last_Name']
    out['PID']['GivenName'] = row['APM_Patient_First_Name']
    out['PID']['MiddleInitialOrName'] = row['APM_Patient_MI']

    # Format the patient's DOB
    out['PID']['DateTimeOfBirth'] = date_change_format(row['APM_Patient_DOB'], "yyyy-MM-dd", "yyyyMMdd")
    out['PID']['AdministrativeSex'] = row['APM_Patient_Sex']

    # IAM Segment processing based on Allergy
    if (row['Status'] == "Active" and len(row['Name']) > 0 and "no known" not in str_to_lower(row['Name'])):
        sIAMseg = f"{str_to_lower(row['Type']).replace(' ', '')}^{row['Type']}|"

        if len(str_trim(row['RXNormCode'])) > 0:
            sIAMseg += f"{str_trim(row['RXNormCode'])}^{str_trim(row['Name'])}^RXN|"
        elif len(str_trim(row['UNIICode'])) > 0:
            sIAMseg += f"{str_trim(row['UNIICode'])}^{str_trim(row['Name'])}^UNII|"
        else:
            sIAMseg += f"^{str_trim(row['Name'])}^NAME|"

        sIAMseg += f"{row['Reaction1Severity']}|" if row['Reaction1Severity'] != "No Severity" else "|"

        sReactions = ""
        for j in range(1, 6):
            reaction = row.get(f'Reaction{j}', "No Reactions")
            if reaction != "No Reactions":
                if len(sReactions) > 0:
                    sReactions += f"~{reaction}^{reaction}"
                else:
                    sReactions = f"{reaction}^{reaction}"
        
        sIAMseg += f"{sReactions}|A||||NTL001^{str_trim(row['Name'])}^EXTELG|"

        est_react_date = str_trim(row['EstimatedReactionDate'])
        if len(est_react_date) > 0 and est_react_date[:2] != "21" and est_react_date[:4] != "1900":
            sIAMseg += f"{date_change_format(est_react_date[:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd')}||"
            iDateIndex = int(date_change_format(est_react_date[:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd'))
        else:
            sIAMseg += f"{date_change_format(row['RecordedDTTM'][:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd')}||"
            iDateIndex = int(date_change_format(row['RecordedDTTM'][:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd'))

        sIAMseg += f"{date_change_format(row['RecordedDTTM'][:19], 'yyyy-MM-dd HH:mm:ss', 'HL7')}|"
        sIAMseg += f"{row['RecordedbyLastName']}^{row['RecordedbyFirstName']}||||"

        # Check for annotations and actions
        sAnnotations = row['Annotation'] if len(row['Annotation']) > 0 and "allergy:" not in str_to_lower(row['Annotation']) and row['Annotation'] != "NULL" else ""
        sActTaken = row['ActionTaken'] if len(row['ActionTaken']) > 0 and row['ActionTaken'] != "NULL" else ""

        # Collecting IAM data
        if not bFound:
            saAllergen.append(sAllIndex.upper())
            iaOnsetDTTM.append(iDateIndex)
            saIAM.append(sIAMseg)
            saAnnotate.append(sAnnotations)
            saActTaken.append(sActTaken)
            iIAMcntr += 1

# Implementing the full HL7 message generation code and running it on the dataset

def convert_to_hl7(df):
    hl7_messages = []

    for row_index, row in df.iterrows():
        # MSH Segment
        hl7_message = []
        hl7_message.append(f"MSH|^~\\&|{mshDefault_SendingApp_TW}|{defaultSendingFac}|{mshDefault_ReceivingApp}|{mshDefault_ReceivingFac}|{date_change_format(row['CreateDTTM'], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMddHHmmss')}||ADT^A60|{row['APM_Patient_ID']}|{mshDefault_ProcessingID}|{mshDefault_VersionID}|")

        # PID Segment
        hl7_message.append(f"PID|||{row['APM_Patient_ID']}^^^MRN||{row['APM_Patient_Last_Name']}^{row['APM_Patient_First_Name']}^{row['APM_Patient_MI']}||{date_change_format(row['APM_Patient_DOB'], 'yyyy-MM-dd', 'yyyyMMdd')}|{row['APM_Patient_Sex']}|||{row['APM_Patient_Street1']}^{row['APM_Patient_Street2']}^{row['APM_Patient_City']}^{row['APM_Patient_State']}^{row['APM_Patient_Zip_Code']}|{row['Patient_Home_Phone']}|{row['Patient_Work_Phone']}|{row['Patient_Email']}")

        # IAM Segment
        if (row['Status'] == "Active" and len(row['Name']) > 0 and "no known" not in str_to_lower(row['Name'])):
            sIAMseg = f"IAM|1|{str_to_lower(row['Type']).replace(' ', '')}^{row['Type']}|"

            if len(str_trim(row['RXNormCode'])) > 0:
                sIAMseg += f"{str_trim(row['RXNormCode'])}^{str_trim(row['Name'])}^RXN|"
            elif len(str_trim(row['UNIICode'])) > 0:
                sIAMseg += f"{str_trim(row['UNIICode'])}^{str_trim(row['Name'])}^UNII|"
            else:
                sIAMseg += f"^{str_trim(row['Name'])}^NAME|"

            sIAMseg += f"{row['Reaction1Severity']}|" if row['Reaction1Severity'] != "No Severity" else "|"

            sReactions = ""
            for j in range(1, 6):
                reaction = row.get(f'Reaction{j}', "No Reactions")
                if reaction != "No Reactions":
                    if len(sReactions) > 0:
                        sReactions += f"~{reaction}^{reaction}"
                    else:
                        sReactions = f"{reaction}^{reaction}"

            sIAMseg += f"{sReactions}|A||||NTL001^{str_trim(row['Name'])}^EXTELG|"

            est_react_date = str_trim(row['EstimatedReactionDate'])
            if len(est_react_date) > 0 and est_react_date[:2] != "21" and est_react_date[:4] != "1900":
                sIAMseg += f"{date_change_format(est_react_date[:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd')}||"
            else:
                recorded_dttm_str = str(row['RecordedDTTM']) if not isinstance(row['RecordedDTTM'], str) else row['RecordedDTTM']
                sIAMseg += f"{date_change_format(recorded_dttm_str[:19], 'yyyy-MM-dd HH:mm:ss', 'yyyyMMdd')}||"

            sIAMseg += f"{date_change_format(recorded_dttm_str[:19], 'yyyy-MM-dd HH:mm:ss', 'HL7')}|"
            sIAMseg += f"{row['RecordedbyLastName']}^{row['RecordedbyFirstName']}||||"

            hl7_message.append(sIAMseg)

        # Join the message parts and append to the list of messages
        hl7_messages.append("\n".join(hl7_message))

    return "\n\n".join(hl7_messages)


# Call the function again and save the HL7 output to a text file
hl7_output = convert_to_hl7(df_fixed)

# Save the HL7 output to a text file
output_file_path = '/mnt/data/hl7_output_final.txt'
with open(output_file_path, 'w') as f:
    f.write(hl7_output)

output_file_path  # Returning the file path for download
